In [1]:
import pandas as pd

https://stackoverflow.com/questions/73693154/pandas-to-extract-directory-data-into-tabular-form

In [2]:
data = """dn: cn=teja_rafoo,ou=Projects,ou=People,ou=LLDI,o=LLC
costCenter: KR000001
LLCResponsible: cn=llcq05075,ou=Personal,ou=People,ou=LLDI,o=LLC
fullName: Entertainment account
LLCAutoSudoRole: cn=teja_rafoo,ou=Auto,ou=Sudoers,ou=Global,o=Services
LLCSecurityLevel: 0
LLCSecDom: cn=LLC,ou=SecurityDomains,ou=LLDI,o=LLC
LLCEndDate: 20000000
mail: jhaboo.udalt@LLC.com
LLCObjectType: ProjectAccount
LLCHomeDirectory: nisMapName=auto.home,ou=LLC_EMEA_NL-LDC01,ou=Locations,ou=LLDI,o=LLC#0#Quality=ScratchHP,NisMap=LLC4008:/proj/LLC4008_teja_rafoo_cache/q,Quota=20,Id=cache
LLCHomeDirectory: nisMapName=auto.home,ou=LLC_EMEA_NL-LDC01,ou=Locations,ou=LLDI,o=LLC#0#Quality=BusinessCriticalHP,Quota=50,NisMap=LLC3008:/proj/LLC3008_teja_rafoo/q
LLCSecurityDomain: cn=LLC,ou=SecurityDomains,ou=Global,o=Services
loginShell: /bin/bash
sn: teja_rafoo
cn: teja_rafoo
LLCStartDate: 20200510
objectClass: Top
objectClass: inetOrgPerson
objectClass: LLCUserAuxClass
objectClass: organizationalPerson
objectClass: Person
LLCPrimaryGroup: cn=teja_rafoo,ou=Generic,ou=Group,ou=LLDI,o=LLC
homeDirectory: /home/teja_rafoo
LLCRequestorGroup: cn=Default,ou=Requestors,ou=Master,o=Services
gecos: Entertainment account
gidNumber: 2365
LLCAutoSudoEntitledUser: cn=llcmgr,ou=Functional,ou=People,ou=LLDI,o=LLC
uidNumber: 9678
LLCLocalStatus: Active
uid: teja_rafoo

dn: cn=chourya_rafoo,ou=Projects,ou=People,ou=LLDI,o=LLC
costCenter: KR000002
LLCResponsible: cn=llcq05075,ou=Personal,ou=People,ou=LLDI,o=LLC
fullName: Entertainment account
LLCAutoSudoRole: cn=chourya_rafoo,ou=Auto,ou=Sudoers,ou=Global,o=Services
LLCSecurityLevel: 0
LLCSecDom: cn=LLC,ou=SecurityDomains,ou=LLDI,o=LLC
LLCEndDate: 20000000
mail: jhaboo.udalt@LLC.com
LLCObjectType: ProjectAccount
LLCHomeDirectory: nisMapName=auto.home,ou=LLC_EMEA_NL-LDC02,ou=Locations,ou=LLDI,o=LLC#0#Quality=ScratchHP,NisMap=LLC4008:/proj/LLC4008_chourya_rafoo_cache/q,Quota=20,Id=cache
LLCHomeDirectory: nisMapName=auto.home,ou=LLC_EMEA_NL-LDC02,ou=Locations,ou=LLDI,o=LLC#0#Quality=BusinessCriticalHP,Quota=50,NisMap=LLC3008:/proj/LLC3008_chourya_rafoo/q
LLCSecurityDomain: cn=LLC,ou=SecurityDomains,ou=Global,o=Services
loginShell: /bin/bash
sn: chourya_rafoo
cn: chourya_rafoo
LLCStartDate: 20200510
objectClass: Top
objectClass: inetOrgPerson
objectClass: LLCUserAuxClass
objectClass: organizationalPerson
objectClass: Person
LLCPrimaryGroup: cn=chourya_rafoo,ou=Generic,ou=Group,ou=LLDI,o=LLC
homeDirectory: /home/chourya_rafoo
LLCRequestorGroup: cn=Default,ou=Requestors,ou=Master,o=Services
gecos: Entertainment account
gidNumber: 2365
LLCAutoSudoEntitledUser: cn=llcmgr,ou=Functional,ou=People,ou=LLDI,o=LLC
uidNumber: 9678
LLCLocalStatus: Active
uid: chourya_rafoo"""

In [3]:
def generate(file):

    with open(file, "r") as handler:
        text = handler.read()

    records = text.split("\n\n")
    for record in records:
        
        result = {}
        for line in record.split("\n"):
            
            key, value = line.split(": ")
            if key in result:
                result[key] += ";" + value
            else:
                result[key] = value
                
        yield result

In [4]:
frame = pd.DataFrame(generate("llc-data.txt"))
frame

,dn,costCenter,LLCResponsible,fullName,LLCAutoSudoRole,LLCSecurityLevel,LLCSecDom,LLCEndDate,mail,LLCObjectType,...,objectClass,LLCPrimaryGroup,homeDirectory,LLCRequestorGroup,gecos,gidNumber,LLCAutoSudoEntitledUser,uidNumber,LLCLocalStatus,uid
0,"cn=teja_rafoo,ou=Projects,ou=People,ou=LLDI,o=LLC",KR000001,"cn=llcq05075,ou=Personal,ou=People,ou=LLDI,o=LLC",Entertainment account,"cn=teja_rafoo,ou=Auto,ou=Sudoers,ou=Global,o=S...",0,"cn=LLC,ou=SecurityDomains,ou=LLDI,o=LLC",20000000,jhaboo.udalt@LLC.com,ProjectAccount,...,Top;inetOrgPerson;LLCUserAuxClass;organization...,"cn=teja_rafoo,ou=Generic,ou=Group,ou=LLDI,o=LLC",/home/teja_rafoo,"cn=Default,ou=Requestors,ou=Master,o=Services",Entertainment account,2365,"cn=llcmgr,ou=Functional,ou=People,ou=LLDI,o=LLC",9678,Active,teja_rafoo
1,"cn=chourya_rafoo,ou=Projects,ou=People,ou=LLDI...",KR000002,"cn=llcq05075,ou=Personal,ou=People,ou=LLDI,o=LLC",Entertainment account,"cn=chourya_rafoo,ou=Auto,ou=Sudoers,ou=Global,...",0,"cn=LLC,ou=SecurityDomains,ou=LLDI,o=LLC",20000000,jhaboo.udalt@LLC.com,ProjectAccount,...,Top;inetOrgPerson;LLCUserAuxClass;organization...,"cn=chourya_rafoo,ou=Generic,ou=Group,ou=LLDI,o...",/home/chourya_rafoo,"cn=Default,ou=Requestors,ou=Master,o=Services",Entertainment account,2365,"cn=llcmgr,ou=Functional,ou=People,ou=LLDI,o=LLC",9678,Active,chourya_rafoo


In [9]:

frame["responsible"] = frame["LLCResponsible"].str.extract("cn=([\w]*)")
frame["location"] = frame["LLCHomeDirectory"].str.extract("ou=([\w_\-]*)")
frame["directory"] = frame["LLCHomeDirectory"].str.findall("NisMap=\w+:([\w_\-/]*)")

In [10]:
frame.loc[0,["LLCHomeDirectory"]].values

array(['nisMapName=auto.home,ou=LLC_EMEA_NL-LDC01,ou=Locations,ou=LLDI,o=LLC#0#Quality=ScratchHP,NisMap=LLC4008:/proj/LLC4008_teja_rafoo_cache/q,Quota=20,Id=cache;nisMapName=auto.home,ou=LLC_EMEA_NL-LDC01,ou=Locations,ou=LLDI,o=LLC#0#Quality=BusinessCriticalHP,Quota=50,NisMap=LLC3008:/proj/LLC3008_teja_rafoo/q'],
      dtype=object)

In [11]:
print(frame[["costCenter", "responsible", "location", "directory", "fullName", "mail"]])

  costCenter responsible           location  \
0   KR000001   llcq05075  LLC_EMEA_NL-LDC01   
1   KR000002   llcq05075  LLC_EMEA_NL-LDC02   

                                           directory               fullName  \
0  [/proj/LLC4008_teja_rafoo_cache/q, /proj/LLC30...  Entertainment account   
1  [/proj/LLC4008_chourya_rafoo_cache/q, /proj/LL...  Entertainment account   

                   mail  
0  jhaboo.udalt@LLC.com  
1  jhaboo.udalt@LLC.com  


In [12]:
print(frame.explode("directory")[["costCenter", "responsible", "directory"]])

  costCenter responsible                            directory
0   KR000001   llcq05075     /proj/LLC4008_teja_rafoo_cache/q
0   KR000001   llcq05075           /proj/LLC3008_teja_rafoo/q
1   KR000002   llcq05075  /proj/LLC4008_chourya_rafoo_cache/q
1   KR000002   llcq05075        /proj/LLC3008_chourya_rafoo/q
